In [79]:
os.listdir("./../data")

['Alphabet.pdf', 'Nvidia.pdf', 'Tesla.pdf', 'Uber.pdf']

In [74]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
##Langsmith Tracking
os.environ["LANGCHAIN_TRACING"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an assistant. Please respond to the user query."),
        ("user","Question:{question}")
    ]
)

from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader
loader = PyPDFDirectoryLoader("./../data")
text_docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(text_docs)

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

In [65]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

In [75]:
db = FAISS.from_documents(documents,embeddings)
db

In [67]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002118C311180>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002118C311D50>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [68]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
You will be reward you positvely if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}""")

In [69]:
#Chain intro - Stuff Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [76]:
retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002118C312D10>)

In [77]:
#retriever chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver,document_chain)

In [78]:
response = retrieval_chain.invoke({"input":"Should I invest in Uber next year according to it's current year performance?"})
response['answer']

"Based on the provided context, here's a step-by-step analysis to help you make an informed decision:\n\n1. **Revenue growth**: Uber's revenue has increased by 17% year-over-year, which is a positive sign. However, it's essential to consider the reasons behind this growth. The company mentions that mobility revenue increased due to a 31% year-over-year growth in Mobility Gross Bookings, while delivery revenue grew 14% year-over-year. Freight revenue, however, decreased by 25% year-over-year, partially offsetting the growth.\n2. **Net income**: Uber's net income attributable to Uber Technologies, Inc. was $1.9 billion, which includes a pre-tax unrealized gain on debt and equity securities. This gain is primarily related to changes in the fair value of their equity securities. Adjusted EBITDA was $4.1 billion, growing $2.3 billion year-over-year.\n3. **Adjusted EBITDA**: The company's Adjusted EBITDA improved significantly, growing 137% year-over-year. This suggests that Uber's operation